In [38]:
import json 
import gzip 
def parse(path): 
    g = gzip.open(path, 'r') 
    for l in g: 
        yield eval(l)

In [39]:

import nltk
import gensim
        
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

def reviews():            
    r_i = 0
    review_item_iterator = iter(parse("../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat.json.gz"))
    for it in range(1000000): #range(1000000):
        if it%10000==0:
            print("it="+str(it))
        review_item = next(review_item_iterator)
        r_i = r_i + 1
        if r_i%10000==0:
            print(str(r_i))
            print(preprocess_sentence)
        if "Cell Phones" in review_item["categories"]:
            reviewText = review_item["reviewText"]
            s_i = 0  
            for sentence in sent_detector.tokenize(reviewText):
                s_i = s_i + 1

                #label = review_item["asin"] + "_" + review_item["reviewerID"] + "_" + str(s_i)
                preprocess_sentence = gensim.utils.simple_preprocess(sentence)

                yield preprocess_sentence
                

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def nextTokenizedSentence():
    yield [token for token in tokens if token not in stop and token not in exclude]



In [40]:
def nextTaggedSentence():
    for tokenizedSentence in reviews():
        #print(tokenizedSentence)
        tagged = nltk.pos_tag(tokenizedSentence)
        yield tagged

In [41]:
def traverse(t,noun_phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            [noun_phrases.append(word) for (word,pos) in t.leaves() if pos.startswith("N")]
            #print(statement)
            #noun_phrases = noun_phrases.append(noun_phrase)
        # Now we know that t.node is defined
        #print('(', t.label(), end=" ")
        for child in t:
            traverse(child,noun_phrases)
        #print(')', end=" ")
    return noun_phrases



In [42]:
def nextNounPhrase( ):
    for taggedSentence in nextTaggedSentence():
        grammar = r"""
          NOUN_PHRASE:  {<CD|IN|DT|JJ|PP|RB|\$>*<N.*>+<V.*>*<CD|IN|DT|JJ|PP|RB|\$>*<N.*>*}  
            """
        cp = nltk.RegexpParser(grammar)
        if taggedSentence:
            noun_phrase_tagged = cp.parse(taggedSentence)
            noun_phrases=[]
            nouns=traverse(noun_phrase_tagged,noun_phrases)
            #print(nouns)
            for noun in nouns:
                yield noun
 

import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
review_source = "amazon" #aliexpress

if review_source == "aliexpress":
    my_collection = mydb['reviews']
    doc_complete = [review_doc["buyerTranslationFeedback"] 
                    for review_doc in my_collection.find() 
                    if "buyerTranslationFeedback" in review_doc.keys() ]
else:
    my_collection = mydb['amazon_product_reviews']
    doc_complete = [review_doc["reviewText"] 
                    for review_doc in my_collection.find({ "categories" : "Cell Phones" }) ]
print("count of reviews: "+ str(len(doc_complete)))



import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
review_source = "amazon" #aliexpress

if review_source == "aliexpress":
    my_collection = mydb['reviews']
    cursor = my_collection.find() 
    textFieldName = "buyerTranslationFeedback"
else:
    my_collection = mydb['amazon_product_reviews']
    cursor = my_collection.find({ "categories" : "Cell Phones" })
    textFieldName = "reviewText"


In [45]:
from nltk import FreqDist

fdist = FreqDist(nextNounPhrase()) 
frequent_words_freq = fdist.most_common(1000)
print(frequent_words_freq )
#fdist.plot()


it=0
10000
['bye', 'bye', 'tracfone', 'it', 'hasn', 'been', 'fun']
it=10000
20000
['you', 'will', 'also', 'need', 'to', 'provide', 'them', 'part', 'sn', 'emei', 'your', 'mobile', 'acct', 'and', 'last', 'of', 'your', 'ssn', 'it', 'is', 'thus', 'recommended', 'to', 'steer', 'clear', 'of', 'this', 'device', 'because', 'its', 'resale', 'value', 'is', 'nill']
it=20000
30000
[]
it=30000
40000
['until', 'then', 'this', 'is', 'perfect', 'for', 'what', 'want']
it=40000
50000
['knowing', 'what', 'know', 'now', 'would', 'still', 'buy', 'this', 'exact', 'phone', 'again', 'if', 'anything', 'happened', 'to', 'the', 'one', 'own', 'now', 'can', 'resist', 'shiny', 'pink', 'things']
it=50000
60000
['amazon', 'was', 'kind', 'enough', 'deduct', 'the', 'cost', 'of', 'that', 'free', 'super', 'saver', 'shipping', 'from', 'my', 'refund', 'as', 'well', 'as', 'making', 'me', 'pay', 'the', 'cost', 'of', 'shipping', 'this', 'worthless', 'phone', 'shaped', 'piece', 'of', 'plastic', 'back', 'to', 'them']
it=60000
7

In [55]:
import gensim
model = gensim.models.Word2Vec.load('../download/word2vec/myamazonmodel.word2vec')

In [57]:
frequent_words= [w for (w,c) in frequent_words_freq]
#print(frequent_words)
import numpy as np
np_frequent_words = np.asarray([w for (w,c) in frequent_words_freq])
np_frequent_words_vector = np.asarray([model.wv[w] for w in np_frequent_words])
print(np_frequent_words_vector)

[[-1.0480926  -1.02450836 -0.95549756 ...,  0.02163913  0.16548625
   0.51798493]
 [-3.06724215 -3.56221414 -0.84086907 ...,  4.36849737 -0.86245358
  -1.76358771]
 [ 0.65128207 -2.39055872  0.16581805 ..., -1.56974447  2.98144603
   3.35234666]
 ..., 
 [ 0.6520279   0.55668253  0.72520012 ..., -2.45997357 -3.07514453
  -0.63901836]
 [-0.70004165 -1.16438103  1.23163009 ...,  0.31264901  1.23521781
  -0.34675583]
 [-1.30441892 -0.93372566  1.36454558 ...,  1.21297908 -0.83029789
  -0.11228187]]


In [58]:
import nltk
from nltk.cluster.kmeans import KMeansClusterer
nc = 10
data = np_frequent_words_vector

kclusterer = KMeansClusterer(nc, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(data, assign_clusters=True)

In [59]:
print(assigned_clusters)
np_assigned_clusters = np.asarray(assigned_clusters)

[2, 8, 1, 6, 6, 2, 4, 8, 1, 0, 8, 7, 0, 6, 8, 5, 0, 1, 3, 4, 1, 2, 5, 7, 9, 6, 5, 6, 6, 1, 8, 5, 2, 6, 5, 4, 8, 8, 0, 6, 6, 7, 5, 2, 0, 5, 9, 9, 0, 1, 8, 9, 4, 5, 0, 2, 6, 3, 5, 9, 4, 8, 3, 0, 5, 0, 5, 3, 4, 6, 5, 9, 9, 8, 0, 4, 4, 6, 1, 4, 0, 7, 5, 3, 9, 1, 0, 5, 2, 3, 5, 9, 3, 0, 3, 2, 0, 5, 0, 9, 2, 5, 5, 6, 5, 5, 9, 7, 5, 5, 6, 0, 2, 0, 6, 5, 2, 3, 2, 4, 0, 8, 8, 4, 2, 5, 5, 8, 7, 4, 4, 4, 2, 1, 5, 9, 9, 8, 1, 5, 9, 9, 5, 0, 5, 2, 8, 8, 0, 2, 1, 3, 4, 0, 2, 4, 9, 0, 8, 9, 0, 1, 7, 4, 5, 1, 5, 1, 3, 2, 3, 4, 5, 3, 3, 8, 9, 0, 2, 5, 4, 3, 6, 4, 8, 6, 5, 8, 8, 7, 5, 8, 5, 0, 4, 7, 6, 4, 0, 2, 3, 3, 5, 8, 4, 5, 5, 5, 1, 9, 9, 5, 5, 5, 5, 7, 8, 2, 5, 5, 9, 1, 2, 0, 2, 5, 3, 8, 4, 5, 3, 2, 7, 1, 0, 4, 1, 1, 5, 6, 8, 3, 4, 8, 5, 7, 3, 5, 2, 5, 0, 9, 2, 8, 9, 8, 5, 1, 5, 5, 1, 5, 4, 4, 4, 5, 0, 7, 1, 0, 8, 4, 3, 4, 2, 7, 8, 0, 6, 5, 1, 3, 9, 4, 6, 9, 5, 4, 7, 4, 5, 7, 9, 4, 0, 5, 8, 7, 5, 3, 0, 7, 5, 5, 1, 7, 6, 7, 8, 8, 2, 3, 8, 7, 1, 1, 7, 9, 0, 2, 2, 9, 0, 2, 8, 6, 5, 8, 7, 2, 9, 7, 9, 

In [60]:
import pandas as pd
#cluster_sentences_join = [" ".join(s_list) for s_list in cluster_sentences]
result = pd.DataFrame({ 'cluster' : assigned_clusters,   'feature' : np_frequent_words})
print(result.head())

   cluster  feature
0        2    phone
1        8   camera
2        1   phones
3        6  battery
4        6     time


In [61]:
writer = pd.ExcelWriter('clustered_features.xlsx')
result.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
features=  [ {"feature" : key} for key in frequent_words]
print(features)

import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
my_collection = mydb['features']
try:
    mydb.features.drop()
finally:
    result = mydb.features.create_index([('feature', pymongo.ASCENDING)],unique=True)

my_collection.insert_many(features)

In [64]:
print(model.wv.most_similar("thing"))

[('downfall', 0.6178915500640869), ('drawback', 0.5932508707046509), ('downside', 0.5874196887016296), ('complaint', 0.5643346309661865), ('gripe', 0.553368091583252), ('quirk', 0.5493331551551819), ('disadvantage', 0.5491840839385986), ('downer', 0.5404207110404968), ('criticism', 0.529247522354126), ('shortcoming', 0.5244871973991394)]
